In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.
last_date = session.query(func.max(Measurement.date)).first()
print(last_date)

#Converting date to integer and breaking out by year, month, day
last_date2=str(last_date)
year = int(last_date2[2]+ last_date2[3]+last_date2[4]+last_date2[5])
print(year)
month = int(last_date2[7]+last_date2[8])
print(month)
day = int(last_date2[10]+last_date2[11])
print(day)

('2017-08-23',)
2017
8
23


In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
maxdate = dt.date(year, month, day)

print(prcp_list)

# Calculate the date one year from the last date in data set.
query_date = dt.date(year, month, day) - dt.timedelta(days=365)
print("Query Date: ", query_date)

# Perform a query to retrieve the data and precipitation scores
prcp_list = []   
prcp_list = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > query_date).filter(Measurement.date <= maxdate).\
    all()

# Save the query results as a Pandas DataFrame and set the index to the date column
prcpdf = pd.DataFrame(prcp_list)
prcpdf['date'] = pd.to_datetime(prcpdf['date'])
prcpdf.head()

# Sort the dataframe by date
prcpdf2 = prcpdf.set_index('date')
prcpdf2.rename(columns = {'prcp': 'precipitation'}, inplace=True)
prcpdf2.head()

# Use Pandas Plotting with Matplotlib to plot the data

prcpdf2.plot(figsize=(15, 8),sort_columns=True,rot=50,use_index=True,legend=True)
plt.xlabel('Date')
plt.ylabel("Precipitation")
plt.title("Precipitation Measurements for 8/24/16-8/23/17", fontsize=20)
plt.savefig('barplot1')
plt.show()


NameError: name 'prcp_list' is not defined

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcpdf2.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
stationcount = []
stationcount = session.query(Station.station).count()
print(stationcount)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
stationhits = []   
stationhits = session.query(Measurement.station, Station.name, func.count(Measurement.station)).\
filter(Measurement.station == Station.station).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
stationhits

mostactive = stationhits[0][1]
mostactivecount = stationhits[0][2]
print(f'The most active station is {mostactive} with {mostactivecount} records.')

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
mostsactivestats = []   
mostsactivestats = session.query(func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
filter(Measurement.station == Station.station).filter(Station.name==mostactive).all()
print(mostsactivestats)
lowesttemp = mostsactivestats[0][0]
highesttemp = mostsactivestats[0][1]
avgtemp = mostsactivestats[0][2]
print(f'For {mostactive}, the lowest temperature is {lowesttemp}. The highest temp is {highesttemp}. The average temp is {avgtemp}.')

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
hist_list = []   
hist_list = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == stationhits[0][0]).filter(Measurement.date > query_date).\
    filter(Measurement.date <= maxdate).\
    all()
hist_list

#adding query to dataframe
hist_df = pd.DataFrame(hist_list)
hist_df.head()
#converting date to datetime
hist_df['date'] = pd.to_datetime(hist_df['date'])
hist_df.head()

hist_temps=hist_df['tobs']
plt.hist(hist_temps, bins=12, color='xkcd:azure')
plt.title("Temperature Observations for Most Active Station for 8/24/16-8/23/17", fontsize=20)
plt.ylabel('Frequency', fontsize=16)
labels = ['tobs']
plt.legend(labels)
plt.savefig('histogram1')
plt.show()

# Close session

In [ ]:
# Close Session
session.close()